# Setup

In [ ]:
# Install all necessary packages
using Distributed
@everywhere using Pkg; @everywhere Pkg.activate(".")
Pkg.instantiate()
using EffectsArmswingGaitAsymmetryStability, Biomechanics, ProgressMeter

In [ ]:
# Add workers for parallel processing
prs = addprocs(;topology=:master_worker, exeflags=["-O3", "--project=@."])

# Load relevant code on all processes
@everywhere using EffectsArmswingGaitAsymmetryStability

In [ ]:
trials = readtrials("data");

In [ ]:
[ (Subject=r[1], numtrials=length(r[2])) for r in [ (sub, findall(t -> t.subject == sub, trials)) for sub in 1:15 ] ]

## Main analysis

In [ ]:
# Setup progressmeter and lock
pdesc = "Processing data... "
p = Progress(length(trials)+1; desc=pdesc, barglyphs=BarGlyphs("[=>.]"))
uplock = ReentrantLock()

In [ ]:
# Update the progressmeter in a thread-safe manner
@everywhere function updateprogress()
    lock(uplock)
    next!(p)
    unlock(uplock)
    nothing
end

# This is used by the workers
@everywhere function analyzeandupdate(trial)
    numstrides = 125
    atrial = analyzetrial(trial, numstrides)
    
    # Tell the master process to update the progressmeter
    remotecall_wait(updateprogress,1)
    return atrial
end

In [ ]:
# Analyze all trials, don't bail on error
next!(p)
analyzedtrials = pmap(analyzeandupdate, trials; on_error=identity)
finish!(p)

In [ ]:
# Check to see if any perturbations failed (to go back and see why they failed)
badtrials = findall(x -> !isa(x, AnalyzedSegment), analyzedtrials)
if !isempty(badtrials)
    @show badtrials
    @show analyzedtrials[badtrials]
#     variables = [
#         :lambdaS,
#         :left_steplavg,
#         :left_steplstd,
#         :right_steplavg,
#         :right_steplstd,
#         :stepwavg,
#         :stepwstd,
#         :rsho_avgrom,
#         :lsho_avgrom,
#         :rsho_stdrom,
#         :lsho_stdrom,
#         :swingasym,
#         :stepasym_spatial,
#         :stepasym_temporal,
#         :msdcrp_lsho_rhip,
#         :msdcrp_rsho_lhip
#     ]
#     # NAN the bad trials
#     analyzedtrials[badtrials] = [ AnalyzedSegment(Segment(trial, Dict{Symbol,Vector}(), SteadyStateSeg()), 
#                                                           Dict{Symbol,Any}(( (var, NaN) for var in variables ))) 
#                                   for trial in trials[badtrials] ]
end

In [ ]:
# We don't need the other workers anymore
rmprocs(prs)

# Generate figure of temporal step asymmetry

In [ ]:
using PlotlyJS, ORCA, Statistics

In [ ]:
means = Dict( sym => Dict( arms => mean(at -> at.results[:stepasym_temporal], 
                                        filter(at -> at.s.trial.conds[:arms] === arms &&
                                                     at.s.trial.conds[:sym] === sym &&
                                                     at.s.trial.subject != 1, # Remove outlier
                                               analyzedtrials))
                  for arms in (:held, :norm, :active) ) 
              for sym in (:sym, :asym) )

stds = Dict( sym => Dict( arms => std([ at.results[:stepasym_temporal] for at in
                                      filter(at -> at.s.trial.conds[:arms] === arms &&
                                                   at.s.trial.conds[:sym] === sym &&
                                                   at.s.trial.subject != 1, # Remove outlier
                                             analyzedtrials)]) 
                  for arms in (:held, :norm, :active) ) 
              for sym in (:sym, :asym) )

In [ ]:
sym = bar(;
    x=["Held", "Normal", "Active"],
    y=map(arms -> means[:sym][arms], (:held, :norm, :active)),
    marker_color="#1a6499",
    name="Symmetric"
)
sym["error_y"] = attr(;
    array=map(arms -> stds[:sym][arms], (:held, :norm, :active)),
    thickness=1
)

asym = bar(;
    x=["Held", "Normal", "Active"],
    y=map(arms -> means[:asym][arms], (:held, :norm, :active)),
    marker_color="#e36868",
    name="Asymmetric"
)
asym["error_y"] = attr(;
    array=map(arms -> stds[:asym][arms], (:held, :norm, :active)),
    thickness=1
)

l = Layout(;
    barmode="group",
    bargap=0.3,
    bargroupgap=0.05,
    xaxis_title="Arm Swing Condition",
    yaxis_title="Asymmetry",
    
    legend=attr(;
        x=0.0,
        y=1.0,
        bgcolor="rgba(255, 255, 255, 1)",
        bordercolor="rgba(255, 255, 255, 0)"
    )
)

p = plot([sym, asym], l)

In [ ]:
savefig(p, "Fig1.svg")

## Results printing setup

In [ ]:
using Statistics, Dates, DelimitedFiles

In [ ]:
# Setup loop variables
variables = [
    :lambdaS,
    :left_steplavg,
    :left_steplstd,
    :right_steplavg,
    :right_steplstd,
    :stepwavg,
    :stepwstd,
    :rsho_avgrom,
    :lsho_avgrom,
    :rsho_stdrom,
    :lsho_stdrom,
    :swingasym,
    :stepasym_spatial,
    :stepasym_temporal,
    :msdcrp_lsho_rhip,
    :msdcrp_rsho_lhip
]
armconds = [ :held, :norm, :active ]
shortarms = Dict(
    :held => "_he",
    :norm => "_nr",
    :active => "_ac"
)
symconds = [ :sym, :asym ]
shortsym = Dict(
    :sym => "_sy,",
    :asym => "_as,"
)

subs = 1:15
numsubs = length(subs)
header = 4

# Initialize the results string
results = Vector{String}(undef, 1)
results[1] = "Analysis of stability, coordination, and symmetry during steady-state walking\n"

In [ ]:
# Loop through all variables and conditions
io = IOBuffer()
for vari in eachindex(variables)
    subresults = fill(",", 4)
    R = collect(1:15)
    _r = Array{Float64}(undef, numsubs)
    for arms in eachindex(armconds), symmetry in eachindex(symconds)
        # Only print the variable/condition if it is the first of its type
        subresults[1] *= prod([ arms, symmetry ] .== ones(Int,2)) ? string(variables[vari])*"," : ","
        subresults[2] *= (symmetry == one(Int)) ? string(armconds[arms], ",") : ","
        subresults[3] *= string(symconds[symmetry], ",")
        subresults[4] *= string(variables[vari])*shortarms[armconds[arms]]*shortsym[symconds[symmetry]]

        # Find all trials for this combination of conditions
        relevant = findall(analyzedtrials) do at
            at.s.trial.conds[:arms] == armconds[arms] &&
            at.s.trial.conds[:sym] == symconds[symmetry]
        end

        for sub in subs
            # Find the trial for this particular subject, convert it back to the absolute indices of `analyzedtrials`
            # `findfirst` to return a scalar (we know that there should/will only be 1 result)
            t = relevant[findfirst(at -> at.s.trial.subject == sub, analyzedtrials[relevant])]

            _r[sub] = analyzedtrials[t].results[variables[vari]]
        end
        
        R = [ R _r ]
    end

    writedlm(io, R, ',')

    results = [results; "\n\n"; subresults; String(take!(io))]
end

## Write to file

In [ ]:
(path, io) = mktemp()

for line in results
    println(io, line)
end

close(io)

resfn = abspath("results.csv")

mv(path, resfn; force=true)